# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [4]:
pip install cartopy

  Obtaining dependency information for cartopy from https://files.pythonhosted.org/packages/bb/50/ab5495dcc22587179134cd37e6534212babeda78e86bda311c37c773f086/Cartopy-0.22.0-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for shapely>=1.7 from https://files.pythonhosted.org/packages/9e/39/029c441d8af32ab423b229c4525ce5ce6707318155b59634811a4c56f5c4/shapely-2.0.2-cp311-cp311-win_amd64.whl.metadata
     ---------------------------------------- 0.0/46.5 kB ? eta -:--:--
     ----------------------------------- ---- 41.0/46.5 kB 1.9 MB/s eta 0:00:01
     ---------------------------------------- 46.5/46.5 kB 1.2 MB/s eta 0:00:00
  Obtaining dependency information for pyproj>=3.1.0 from https://files.pythonhosted.org/packages/79/95/eb68113c5b5737c342bde1bab92705dabe69c16299c5a122616e50f1fbd6/pyproj-3.6.1-cp311-cp311-win_amd64.whl.metadata
   ---------------------------------------- 0.0/11.2 MB ? eta -:--:--
   - -------------------------------------- 0.5/11.2 MB 11.6 MB

In [5]:
pip install geoviews

  Obtaining dependency information for geoviews from https://files.pythonhosted.org/packages/72/10/81aaca3edf7d7bfb14a9a17b5effaeb9dcba54ba9703b392bb17241e45ea/geoviews-1.11.0-py2.py3-none-any.whl.metadata
   ---------------------------------------- 0.0/511.2 kB ? eta -:--:--
    --------------------------------------- 10.2/511.2 kB ? eta -:--:--
   ---------------------------- ----------- 368.6/511.2 kB 5.7 MB/s eta 0:00:01
   ---------------------------------------- 511.2/511.2 kB 6.3 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install pyproj

Note: you may need to restart the kernel to use updated packages.


In [1]:
# Dependencies and Setup
import json
import numpy as np
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,cedral,23.8000,-100.7333,16.38,85,91,3.09,MX,1699932594
1,1,waitangi,-43.9535,-176.5597,13.31,89,4,9.38,NZ,1699932595
2,2,ulanhot,46.0833,122.0833,2.57,32,2,2.30,CN,1699932334
3,3,utrik,11.2278,169.8474,28.09,81,100,7.99,MH,1699932594
4,4,opuwo,-18.0607,13.8400,24.51,26,53,0.96,NaN,1699932569


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
all_city_map = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    alpha = 0.5,
    color = 'Country'
)

# Display the map
all_city_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Country,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
city_temp0 = city_data_df.loc[city_data_df['Max Temp']<27,:]
city_temp = city_temp0.loc[city_temp0['Max Temp']>21,:]
city_wind = city_temp.loc[city_temp['Wind Speed']<4.5,:]
city_cloud = city_wind.loc[city_wind['Cloudiness']==0,:]
city_cloud

# Drop any rows with null values
city_ideal = city_cloud.dropna()

# Display sample data
city_ideal

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
160,160,guanica,17.9716,-66.9080,26.21,62,0,3.04,PR,1699932709
217,217,miandrivazo,-19.5167,45.4667,25.06,66,0,1.21,MG,1699932744
227,227,tazacorte,28.6290,-17.9293,23.97,33,0,4.02,ES,1699932751
242,242,sao joao do piaui,-8.3581,-42.2467,24.98,28,0,0.97,BR,1699932761
248,248,cannonvale,-20.2833,148.7000,26.66,52,0,4.18,AU,1699932765
306,306,tahoua,14.8888,5.2692,21.99,21,0,4.12,NE,1699932798
307,307,timbuktu,16.7735,-3.0074,23.33,13,0,4.28,ML,1699932799
335,335,conceicao do araguaia,-8.2578,-49.2647,26.53,48,0,1.87,BR,1699932817
347,347,riacho de santana,-13.6092,-42.9389,23.94,38,0,2.19,BR,1699932824
382,382,livramento do brumado,-13.6431,-41.8406,23.42,45,0,1.39,BR,1699932841


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_ideal.loc[:,['City','Country','Lat','Lng','Humidity']]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
160,guanica,PR,17.9716,-66.9080,62,
217,miandrivazo,MG,-19.5167,45.4667,66,
227,tazacorte,ES,28.6290,-17.9293,33,
242,sao joao do piaui,BR,-8.3581,-42.2467,28,
248,cannonvale,AU,-20.2833,148.7000,52,
306,tahoua,NE,14.8888,5.2692,21,
307,timbuktu,ML,16.7735,-3.0074,13,
335,conceicao do araguaia,BR,-8.2578,-49.2647,48,
347,riacho de santana,BR,-13.6092,-42.9389,38,
382,livramento do brumado,BR,-13.6431,-41.8406,45,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "apiKey":geoapify_key,
    "categories":"accommodation.hotel"
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row['Lat']
    lon = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
guanica - nearest hotel: Copamarina Beach Resort
miandrivazo - nearest hotel: Baobab
tazacorte - nearest hotel: App Leyma
sao joao do piaui - nearest hotel: Hotel Esplanada
cannonvale - nearest hotel: Whitsundays Rainforest Retreat
tahoua - nearest hotel: Hotel Bungalow
timbuktu - nearest hotel: Hotel La Maison
conceicao do araguaia - nearest hotel: No hotel found
riacho de santana - nearest hotel: No hotel found
livramento do brumado - nearest hotel: No hotel found
remire-montjoly - nearest hotel: Complexe Belova


,City,Country,Lat,Lng,Humidity,Hotel Name
160,guanica,PR,17.9716,-66.9080,62,Copamarina Beach Resort
217,miandrivazo,MG,-19.5167,45.4667,66,Baobab
227,tazacorte,ES,28.6290,-17.9293,33,App Leyma
242,sao joao do piaui,BR,-8.3581,-42.2467,28,Hotel Esplanada
248,cannonvale,AU,-20.2833,148.7000,52,Whitsundays Rainforest Retreat
306,tahoua,NE,14.8888,5.2692,21,Hotel Bungalow
307,timbuktu,ML,16.7735,-3.0074,13,Hotel La Maison
335,conceicao do araguaia,BR,-8.2578,-49.2647,48,No hotel found
347,riacho de santana,BR,-13.6092,-42.9389,38,No hotel found
382,livramento do brumado,BR,-13.6431,-41.8406,45,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    alpha =0.8,
    color = 'City',
    hover_cols = ["Hotel Name", "Country"]
)

# Display the map
hotel_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)